# Evaluación Temporal del Modelo NYC Taxi Tips

Este notebook demuestra el comportamiento del modelo de clasificación de propinas a lo largo del tiempo, evaluando su rendimiento en múltiples meses para identificar patrones de degradación y factores que afectan su estabilidad.

## Objetivos:
1. **Carga y evaluación** del modelo en múltiples meses
2. **Automatización** de la evaluación mensual
3. **Visualización** de patrones temporales
4. **Análisis crítico** del rendimiento y recomendaciones

## 1. Configuración Inicial

In [ ]:
# Importaciones necesarias
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Añadir src al path para importar nuestro paquete
current_dir = Path.cwd()
if 'notebook' in str(current_dir):
    src_dir = current_dir.parent / "src"
else:
    src_dir = current_dir / "src"

if src_dir.exists():
    sys.path.insert(0, str(src_dir))
    print(f"✅ Agregado al path: {src_dir}")
else:
    print(f"⚠️  Directorio src no encontrado: {src_dir}")

print(f"📁 Directorio de trabajo: {current_dir}")

In [ ]:
# Importar nuestro paquete de evaluación temporal
from taxi_tips_classifier.temporal_evaluation import TemporalModelEvaluator
from taxi_tips_classifier.model import TaxiTipClassifier

print("✅ Módulos importados correctamente")

## 2. Configuración del Experimento

In [ ]:
# Configuración del experimento
MODEL_PATH = "../random_forest.joblib"  # Ruta al modelo entrenado
SAMPLE_SIZE = 25000  # Muestras por mes para evaluación rápida (None = todos los datos)

# Meses a evaluar (modelo entrenado en enero 2020)
MONTHS_TO_EVALUATE = [
    "2020-01",  # Mes de entrenamiento (baseline)
    "2020-02",  # Febrero
    "2020-03",  # Marzo (inicio COVID-19)
    "2020-04",  # Abril (confinamiento)
    "2020-05",  # Mayo (reapertura gradual)
    "2020-06"   # Junio (nueva normalidad)
]

print(f"🎯 Configuración del experimento:")
print(f"   📁 Modelo: {MODEL_PATH}")
print(f"   📊 Muestras por mes: {SAMPLE_SIZE or 'Todos los datos'}")
print(f"   📅 Meses a evaluar: {len(MONTHS_TO_EVALUATE)}")
print(f"   🗓️  Período: {MONTHS_TO_EVALUATE[0]} - {MONTHS_TO_EVALUATE[-1]}")

## 3. Inicialización del Evaluador Temporal

In [ ]:
# Verificar que el modelo existe
import os
if not os.path.exists(MODEL_PATH):
    print(f"❌ Modelo no encontrado en: {MODEL_PATH}")
    print("💡 Ejecuta primero el script de entrenamiento:")
    print("   python scripts/train_simple.py")
else:
    print(f"✅ Modelo encontrado: {MODEL_PATH}")
    
    # Inicializar el evaluador temporal
    evaluator = TemporalModelEvaluator(model_path=MODEL_PATH)
    print("✅ Evaluador temporal inicializado")

## 4. Evaluación Mensual Automatizada

Aquí ejecutamos la evaluación del modelo en cada mes seleccionado, calculando métricas de rendimiento y recopilando estadísticas.

In [ ]:
# Ejecutar evaluación mensual
print("🚀 Iniciando evaluación temporal...")
print("=" * 50)

# Esta función automatiza la evaluación para múltiples meses
monthly_results = evaluator.evaluate_multiple_months(
    months=MONTHS_TO_EVALUATE,
    sample_size=SAMPLE_SIZE
)

print(f"\n✅ Evaluación completada para {len(monthly_results)} meses")

## 5. Tabla de Resultados Mensuales

Organizamos los resultados en una tabla clara con las métricas principales por mes.

In [ ]:
# Crear tabla de resultados
results_table = evaluator.create_results_table(monthly_results)

print("📊 TABLA DE RESULTADOS MENSUALES")
print("=" * 50)
display(results_table)

# Estadísticas resumidas
f1_scores = [float(score) for score in results_table['F1-Score']]
print(f"\n📈 ESTADÍSTICAS RESUMIDAS:")
print(f"   F1-Score promedio: {np.mean(f1_scores):.4f}")
print(f"   F1-Score mínimo: {np.min(f1_scores):.4f}")
print(f"   F1-Score máximo: {np.max(f1_scores):.4f}")
print(f"   Desviación estándar: {np.std(f1_scores):.4f}")
print(f"   Coeficiente de variación: {(np.std(f1_scores)/np.mean(f1_scores))*100:.2f}%")

## 6. Visualización del Rendimiento Temporal

Creamos gráficos para visualizar patrones, tendencias y anomalías en el rendimiento del modelo.

In [ ]:
# Generar visualizaciones completas
fig = evaluator.plot_temporal_performance(
    results=monthly_results,
    figsize=(16, 12)
)

plt.show()

print("📊 Gráficos generados:")
print("   1. Métricas de rendimiento por mes")
print("   2. Número de muestras por mes")
print("   3. Distribución de propinas altas")
print("   4. F1-Score con intervalos de confianza")

## 7. Análisis de Degradación del Modelo

Analizamos específicamente cómo el rendimiento del modelo cambia respecto al mes de entrenamiento (baseline).

In [ ]:
# Análisis detallado de degradación
degradation_analysis = evaluator.analyze_performance_degradation(monthly_results)

print("🔍 ANÁLISIS DE DEGRADACIÓN DEL MODELO")
print("=" * 50)

baseline = degradation_analysis['baseline_month']
baseline_f1 = degradation_analysis['baseline_f1_score']
max_degradation = degradation_analysis['max_degradation']
most_degraded = degradation_analysis['most_degraded_month']

print(f"📅 Mes baseline (entrenamiento): {baseline}")
print(f"🎯 F1-Score baseline: {baseline_f1:.4f}")
print(f"📉 Degradación máxima: {max_degradation:.4f} ({(max_degradation/baseline_f1)*100:.1f}%)")
print(f"⚠️  Mes más degradado: {most_degraded}")

print(f"\n📊 Cambios mensuales respecto al baseline:")
for month, change_info in degradation_analysis['monthly_changes'].items():
    change = change_info['change_from_baseline']
    percentage = change_info['percentage_change']
    status = "📈" if change >= 0 else "📉"
    print(f"   {status} {month}: {change:+.4f} ({percentage:+.1f}%)")

## 8. Análisis de Tendencias y Correlaciones

In [ ]:
# Análisis de tendencias
trends = degradation_analysis['trends']

print("📈 ANÁLISIS DE TENDENCIAS")
print("=" * 40)

print(f"🎯 Tendencia F1-Score: {trends['f1_trend']}")
print(f"📊 Tendencia tamaño muestras: {trends['sample_size_trend']}")
print(f"💰 Tendencia tasa propinas altas: {trends['high_tip_rate_trend']}")
print(f"📐 Varianza F1-Score: {trends['f1_variance']:.6f}")

print(f"\n🔗 CORRELACIONES:")
corr_sample = trends['correlation_f1_sample_size']
corr_tip_rate = trends['correlation_f1_tip_rate']

print(f"   F1 vs Tamaño muestra: {corr_sample:.3f}")
print(f"   F1 vs Tasa propinas altas: {corr_tip_rate:.3f}")

# Interpretación de correlaciones
if abs(corr_tip_rate) > 0.5:
    direction = "positiva" if corr_tip_rate > 0 else "negativa"
    print(f"   💡 Correlación {direction} fuerte con comportamiento de propinas")

if abs(corr_sample) > 0.5:
    direction = "positiva" if corr_sample > 0 else "negativa"
    print(f"   💡 Correlación {direction} fuerte con tamaño de muestra")

## 9. Visualización Adicional: Evolución Temporal Detallada

In [ ]:
# Crear un gráfico personalizado para mostrar la evolución temporal
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Análisis Temporal Detallado - NYC Taxi Tips Model', fontsize=16, fontweight='bold')

months = sorted(monthly_results.keys())
month_labels = [month.replace('2020-', '') for month in months]

# 1. Evolución de métricas principales
ax1 = axes[0, 0]
metrics = ['f1_score', 'accuracy', 'precision', 'recall']
colors = ['red', 'blue', 'green', 'orange']
for metric, color in zip(metrics, colors):
    values = [monthly_results[month][metric] for month in months]
    ax1.plot(month_labels, values, marker='o', linewidth=2, color=color, label=metric.replace('_', ' ').title())

ax1.set_title('Evolución de Métricas de Rendimiento')
ax1.set_xlabel('Mes (2020)')
ax1.set_ylabel('Score')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0.6, 1.0)

# 2. Cambios relativos al baseline
ax2 = axes[0, 1]
baseline_f1 = monthly_results[months[0]]['f1_score']
relative_changes = [(monthly_results[month]['f1_score'] - baseline_f1) * 100 for month in months]
bars = ax2.bar(month_labels, relative_changes, color=['green' if x >= 0 else 'red' for x in relative_changes])
ax2.set_title(f'Cambio Relativo vs Baseline ({months[0]})')
ax2.set_xlabel('Mes (2020)')
ax2.set_ylabel('Cambio en F1-Score (%)')
ax2.axhline(y=0, color='black', linestyle='-', alpha=0.5)
ax2.grid(True, alpha=0.3)

# 3. Distribución de clases por mes
ax3 = axes[1, 0]
high_tip_counts = [monthly_results[month]['support']['positive_samples'] for month in months]
low_tip_counts = [monthly_results[month]['support']['negative_samples'] for month in months]

x = np.arange(len(month_labels))
width = 0.35
ax3.bar(x - width/2, high_tip_counts, width, label='Propinas Altas', color='gold', alpha=0.8)
ax3.bar(x + width/2, low_tip_counts, width, label='Propinas Bajas', color='lightblue', alpha=0.8)

ax3.set_title('Distribución de Clases por Mes')
ax3.set_xlabel('Mes (2020)')
ax3.set_ylabel('Número de Muestras')
ax3.set_xticks(x)
ax3.set_xticklabels(month_labels)
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Análisis de estabilidad (rolling variance)
ax4 = axes[1, 1]
f1_values = [monthly_results[month]['f1_score'] for month in months]
if len(f1_values) >= 3:
    rolling_std = pd.Series(f1_values).rolling(window=3, center=True).std()
    ax4.plot(month_labels, rolling_std, marker='s', linewidth=2, color='purple')
    ax4.fill_between(month_labels, rolling_std, alpha=0.3, color='purple')
    ax4.set_title('Estabilidad del Modelo (Desv. Estándar Rolling)')
else:
    ax4.bar(month_labels, [np.std(f1_values)] * len(month_labels), color='purple', alpha=0.6)
    ax4.set_title('Variabilidad General del F1-Score')

ax4.set_xlabel('Mes (2020)')
ax4.set_ylabel('Desviación Estándar')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Análisis visual completado")

## 10. Análisis Crítico y Explicación de Resultados

### 🎯 Respuestas a las Preguntas Clave:

#### ¿El modelo mantiene un rendimiento consistente?

In [ ]:
# Análisis de consistencia
f1_scores = [monthly_results[month]['f1_score'] for month in months]
cv_coefficient = (np.std(f1_scores) / np.mean(f1_scores)) * 100
max_drop = max([degradation_analysis['monthly_changes'][month]['degradation'] 
                for month in degradation_analysis['monthly_changes']])

print("🎯 CONSISTENCIA DEL RENDIMIENTO")
print("=" * 40)

if cv_coefficient < 5:
    consistency_level = "MUY CONSISTENTE"
    emoji = "✅"
elif cv_coefficient < 10:
    consistency_level = "MODERADAMENTE CONSISTENTE"
    emoji = "⚠️"
else:
    consistency_level = "INCONSISTENTE"
    emoji = "❌"

print(f"{emoji} El modelo es {consistency_level}")
print(f"   Coeficiente de variación: {cv_coefficient:.2f}%")
print(f"   Caída máxima: {max_drop:.4f} ({(max_drop/baseline_f1)*100:.1f}%)")

if max_drop > 0.05:
    print(f"   ⚠️  ALERTA: Degradación significativa detectada")
elif max_drop > 0.02:
    print(f"   ⚠️  Degradación moderada observada")
else:
    print(f"   ✅ Degradación dentro de límites aceptables")

#### ¿Qué factores podrían explicar la variación en el desempeño?

In [ ]:
print("🔍 FACTORES QUE EXPLICAN LA VARIACIÓN")
print("=" * 45)

# Factor 1: Cambios en el comportamiento de propinas
tip_rates = []
for month in months:
    total = monthly_results[month]['total_samples']
    high_tips = monthly_results[month]['support']['positive_samples']
    tip_rates.append((high_tips / total) * 100)

tip_rate_variance = np.var(tip_rates)
tip_rate_change = abs(tip_rates[-1] - tip_rates[0])

print(f"1. 💰 COMPORTAMIENTO DE PROPINAS:")
print(f"   Tasa propinas altas inicial: {tip_rates[0]:.1f}%")
print(f"   Tasa propinas altas final: {tip_rates[-1]:.1f}%")
print(f"   Cambio total: {tip_rates[-1] - tip_rates[0]:+.1f} puntos porcentuales")
print(f"   Varianza en tasas: {tip_rate_variance:.2f}")

if tip_rate_change > 2:
    print(f"   📊 Factor SIGNIFICATIVO: Gran cambio en comportamiento de propinas")

# Factor 2: Efectos estacionales/temporales (COVID-19)
print(f"\n2. 📅 EFECTOS TEMPORALES (COVID-19):")
covid_months = ['2020-03', '2020-04', '2020-05']
covid_performance = [monthly_results[month]['f1_score'] for month in covid_months if month in monthly_results]
pre_covid = [monthly_results[month]['f1_score'] for month in ['2020-01', '2020-02'] if month in monthly_results]

if covid_performance and pre_covid:
    covid_avg = np.mean(covid_performance)
    pre_covid_avg = np.mean(pre_covid)
    covid_impact = covid_avg - pre_covid_avg
    
    print(f"   F1-Score pre-COVID: {pre_covid_avg:.4f}")
    print(f"   F1-Score durante COVID: {covid_avg:.4f}")
    print(f"   Impacto COVID: {covid_impact:+.4f}")
    
    if abs(covid_impact) > 0.02:
        direction = "negativo" if covid_impact < 0 else "positivo"
        print(f"   📊 Factor SIGNIFICATIVO: Impacto {direction} del COVID-19")

# Factor 3: Tamaño de muestra
sample_sizes = [monthly_results[month]['total_samples'] for month in months]
sample_variance = np.var(sample_sizes)
corr_sample = trends['correlation_f1_sample_size']

print(f"\n3. 📊 TAMAÑO DE MUESTRA:")
print(f"   Correlación F1 vs Tamaño: {corr_sample:.3f}")
print(f"   Varianza en tamaños: {sample_variance:,.0f}")

if abs(corr_sample) > 0.5:
    direction = "positiva" if corr_sample > 0 else "negativa"
    print(f"   📊 Factor RELEVANTE: Correlación {direction} con tamaño de muestra")

# Factor 4: Drift en características
print(f"\n4. 🔄 CONCEPT DRIFT:")
if trends['f1_variance'] > 0.001:
    print(f"   Alta variabilidad (σ²={trends['f1_variance']:.6f}) sugiere drift temporal")
    print(f"   📊 Factor PROBABLE: Cambio en distribución de características")
else:
    print(f"   Baja variabilidad sugiere estabilidad en características")

#### ¿Qué acciones recomendarías para mejorar la robustez del modelo?

In [ ]:
# Generar recomendaciones personalizadas basadas en el análisis
print("💡 RECOMENDACIONES PARA MEJORAR LA ROBUSTEZ")
print("=" * 50)

recommendations = []

# Recomendaciones basadas en degradación
if max_drop > 0.05:
    recommendations.append(
        "🔄 CRÍTICO: Implementar reentrenamiento automático mensual debido a degradación significativa"
    )
    recommendations.append(
        "📊 Establecer alertas cuando F1-score caiga más del 5% respecto al baseline"
    )

# Recomendaciones basadas en variabilidad
if cv_coefficient > 5:
    recommendations.append(
        "📈 Implementar técnicas de regularización temporal para reducir variabilidad"
    )
    recommendations.append(
        "🎯 Considerar ensemble models con diferentes ventanas temporales"
    )

# Recomendaciones basadas en correlaciones
if abs(trends['correlation_f1_tip_rate']) > 0.5:
    recommendations.append(
        "💰 Incorporar características dinámicas del comportamiento de propinas"
    )
    recommendations.append(
        "📊 Monitorear la distribución de propinas como indicador de drift"
    )

# Recomendaciones generales
recommendations.extend([
    "🔍 Implementar monitoreo continuo de data drift usando tests estadísticos",
    "⏰ Establecer ventanas de reentrenamiento adaptativas basadas en degradación",
    "🎛️ Implementar A/B testing para comparar modelos en producción",
    "📋 Crear dashboard de monitoreo con métricas temporales en tiempo real",
    "🧪 Experimentar con domain adaptation techniques para cambios temporales",
    "📅 Incorporar características estacionales (día de semana, mes, festividades)",
    "🏗️ Desarrollar pipeline de feature engineering adaptativo",
    "📈 Implementar early warning system basado en degradación de métricas"
])

# Imprimir recomendaciones numeradas
for i, rec in enumerate(recommendations, 1):
    print(f"{i:2d}. {rec}")

print(f"\n🎯 PRIORIDADES INMEDIATAS:")
if max_drop > 0.05:
    print(f"   1. Reentrenamiento urgente del modelo")
    print(f"   2. Implementación de sistema de alertas")
elif cv_coefficient > 10:
    print(f"   1. Análisis detallado de causas de variabilidad")
    print(f"   2. Implementación de monitoreo continuo")
else:
    print(f"   1. Monitoreo preventivo mensual")
    print(f"   2. Mejoras incrementales en pipeline")

## 11. Resumen Ejecutivo y Conclusiones

In [ ]:
# Generar resumen ejecutivo
print("📋 RESUMEN EJECUTIVO - EVALUACIÓN TEMPORAL")
print("=" * 55)

# Métricas clave
baseline_month = degradation_analysis['baseline_month']
baseline_score = degradation_analysis['baseline_f1_score']
final_month = months[-1]
final_score = monthly_results[final_month]['f1_score']
total_change = final_score - baseline_score

print(f"📊 MÉTRICAS CLAVE:")
print(f"   Período evaluado: {months[0]} - {months[-1]} ({len(months)} meses)")
print(f"   F1-Score inicial: {baseline_score:.4f}")
print(f"   F1-Score final: {final_score:.4f}")
print(f"   Cambio total: {total_change:+.4f} ({(total_change/baseline_score)*100:+.1f}%)")
print(f"   Coeficiente de variación: {cv_coefficient:.2f}%")

# Estado del modelo
print(f"\n🎯 ESTADO DEL MODELO:")
if max_drop < 0.02 and cv_coefficient < 5:
    status = "✅ SALUDABLE"
    action = "Continuar monitoreo regular"
elif max_drop < 0.05 and cv_coefficient < 10:
    status = "⚠️ REQUIERE ATENCIÓN"
    action = "Implementar monitoreo mejorado"
else:
    status = "❌ CRÍTICO"
    action = "Reentrenamiento inmediato requerido"

print(f"   Estado: {status}")
print(f"   Acción recomendada: {action}")

# Factores identificados
print(f"\n🔍 FACTORES PRINCIPALES IDENTIFICADOS:")
key_factors = []

if abs(trends['correlation_f1_tip_rate']) > 0.5:
    key_factors.append("Cambios en comportamiento de propinas")

if tip_rate_change > 2:
    key_factors.append("Variación significativa en distribución de clases")

if trends['f1_variance'] > 0.001:
    key_factors.append("Concept drift temporal")

if 'covid_impact' in locals() and abs(covid_impact) > 0.02:
    key_factors.append("Impacto de eventos externos (COVID-19)")

if not key_factors:
    key_factors.append("Rendimiento estable sin factores críticos identificados")

for i, factor in enumerate(key_factors, 1):
    print(f"   {i}. {factor}")

# Próximos pasos
print(f"\n🚀 PRÓXIMOS PASOS:")
next_steps = [
    "Guardar este análisis como baseline para futuras evaluaciones",
    "Establecer pipeline de monitoreo automatizado",
    "Definir umbrales de alerta basados en estos resultados",
    "Programar reevaluaciones mensuales automáticas"
]

for i, step in enumerate(next_steps, 1):
    print(f"   {i}. {step}")

print(f"\n📅 Evaluación completada: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 55)

## 12. Exportar Resultados

In [ ]:
# Guardar resultados para uso futuro
import os
from datetime import datetime

# Crear directorio de resultados
results_dir = "temporal_evaluation_results"
os.makedirs(results_dir, exist_ok=True)

# Guardar tabla de resultados
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
table_path = os.path.join(results_dir, f"resultados_temporales_{timestamp}.csv")
results_table.to_csv(table_path, index=False)

# Guardar análisis completo
import joblib
analysis_path = os.path.join(results_dir, f"analisis_completo_{timestamp}.joblib")
complete_analysis = {
    'monthly_results': monthly_results,
    'degradation_analysis': degradation_analysis,
    'results_table': results_table,
    'metadata': {
        'model_path': MODEL_PATH,
        'sample_size': SAMPLE_SIZE,
        'months_evaluated': MONTHS_TO_EVALUATE,
        'evaluation_date': datetime.now().isoformat(),
        'consistency_level': consistency_level,
        'cv_coefficient': cv_coefficient,
        'max_degradation': max_drop
    }
}

joblib.dump(complete_analysis, analysis_path)

print(f"💾 RESULTADOS GUARDADOS:")
print(f"   📄 Tabla CSV: {table_path}")
print(f"   📊 Análisis completo: {analysis_path}")
print(f"   📁 Directorio: {results_dir}")

print(f"\n✅ Evaluación temporal completada exitosamente")
print(f"📋 Este notebook demostró el comportamiento del modelo a lo largo del tiempo")
print(f"🎯 Los resultados proporcionan insights valiosos para la mejora continua del modelo")